In [ ]:
from glob import glob
import tensorflow as tf

In [ ]:
def parse_tfrecord_fn(example):
    feature_description = {
        "point_cloud": tf.io.FixedLenFeature([], tf.string),
        "label_cloud": tf.io.FixedLenFeature([], tf.string)
    }
    example = tf.io.parse_single_example(example, feature_description)
    point_cloud = tf.io.parse_tensor(example["point_cloud"], out_type=tf.float32)
    label_cloud = tf.io.parse_tensor(example["label_cloud"], out_type=tf.float32)
    return point_cloud, label_cloud

def augment(point_cloud_batch, label_cloud_batch):
    """Jitter point and label clouds."""
    noise = tf.random.uniform(
        tf.shape(label_cloud_batch), -0.005, 0.005, dtype=tf.float32
    )
    point_cloud_batch += noise[:, :, :3]
    return point_cloud_batch, label_cloud_batch

In [ ]:
raw_dataset = tf.data.TFRecordDataset(glob("../tfrecords/Airplane/train/*.tfrec"))
dataset = raw_dataset.map(parse_tfrecord_fn, num_parallel_calls=tf.data.AUTOTUNE).batch(16)
dataset = dataset.map(augment, num_parallel_calls=tf.data.AUTOTUNE)

In [ ]:
x, y = next(iter(dataset))

In [ ]:
x.shape, y.shape